In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('events').getOrCreate()

In [ ]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as sf
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("events").setMaster("local")
sc = SparkContext.getOrCreate()
sqlc = pyspark.SQLContext(sc)

In [ ]:
readdataframe = spark.read.option("sep","\t").csv("examples.tsv",header='true', inferSchema='true')

In [ ]:
#creating an identifier column for traversing the rows
from pyspark.sql.window import *
from pyspark.sql.functions import row_number
w = Window.orderBy("userid")
result = readdataframe.withColumn("index", row_number().over(w))

In [ ]:
r=result.toPandas()

In [ ]:
#selecting all the unique rows of the dataset
uniqueindexes = [list(x.asDict().values())[0] for x in result.select("index").collect()]

# Extracting the distinct events from the dataset

In [ ]:
#selecting the distinct events from the dataset
distinctevents = [list(x.asDict().values())[0] for x in result.select("event").distinct().collect()]

In [ ]:
#extracting the events from all the rows and finding the unique out of all the extracted events
used1=[]
for j in range(0,len(distinctevents)):
    s=distinctevents[j].split(",")
    for i in range(0,len(s)):
        unique = [x for x in s if x not in used1 and used1.append(x)]

In [ ]:
#sorting the extracted events
sort=sorted(used1)

# Extracting the distinct events from each row of the dataset

In [ ]:
#creating a spark dataframe
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
schema = StructType([
    StructField("rowid",IntegerType(),True)
    
])
event2 = spark.createDataFrame(sc.emptyRDD(), schema)

In [ ]:
#assigning the unique events as columns
from pyspark.sql.functions import lit 
outcols = []
for column in sort:
    if column in event2.columns:
        outcols.append(column)
    else:
        outcols.append(lit(None).cast(StringType()).alias('{0}'.format(column)))

event2 = event2.select(outcols)

In [ ]:
#storing the column names in a variable
c=event2.columns

In [ ]:
#finding the unique events for a particular row using list and storing it as a tuple
event1=[]
unique=[]
list1=[]
label2=[]
for x in uniqueindexes:
    for x in result.where(result.index == x).select("event").collect():
        event1=list(x.asDict().values())
        s=str(event1)[2:-2]#removing the brackets assigned because of list and converting to a string
        t=s.split(",")
        for i in range(0,len(t)):
            used=[]
            unique = [i for i in t if i not in used and used.append(i)]
            u=sorted(used)
            label=[]
            for i in range(0,len(sort)):
                if sort[i] in u:
                    label.append(1)
                else:
                    label.append(0)
        tuple1=tuple(label)
        label2.append(tuple1)

In [ ]:
#creating a dataframe from the obtained list of unique events where 1 means the event is present and 0 indicates absent
import pandas as pd
newdf=pd.DataFrame(data=label2, columns=c)

In [ ]:
r.join(newdf)
